In [41]:
from pyspi.calculator import Calculator
import dill
import pyspi
import pandas as pd
from copy import deepcopy
import numpy as np

In [11]:
with open('./benchmark_calculators/calc_standard_normal.pkl', 'rb') as f:
    calc = dill.load(f)

In [23]:
spi_dict = calc.spis
spi_dict

{'cov_EmpiricalCovariance': <pyspi.statistics.basic.Covariance at 0x7f842be5f1c0>,
 'cov_EllipticEnvelope': <pyspi.statistics.basic.Covariance at 0x7f842be5f0d0>,
 'cov_GraphicalLasso': <pyspi.statistics.basic.Covariance at 0x7f842a6ac8b0>,
 'cov_GraphicalLassoCV': <pyspi.statistics.basic.Covariance at 0x7f8469255310>,
 'cov_LedoitWolf': <pyspi.statistics.basic.Covariance at 0x7f84692551f0>,
 'cov_MinCovDet': <pyspi.statistics.basic.Covariance at 0x7f8469255370>,
 'cov_OAS': <pyspi.statistics.basic.Covariance at 0x7f84692552e0>,
 'cov_ShrunkCovariance': <pyspi.statistics.basic.Covariance at 0x7f8469255220>,
 'cov-sq_EmpiricalCovariance': <pyspi.statistics.basic.Covariance at 0x7f8469255070>,
 'cov-sq_EllipticEnvelope': <pyspi.statistics.basic.Covariance at 0x7f84692550d0>,
 'cov-sq_GraphicalLasso': <pyspi.statistics.basic.Covariance at 0x7f8469255430>,
 'cov-sq_GraphicalLassoCV': <pyspi.statistics.basic.Covariance at 0x7f84692553d0>,
 'cov-sq_LedoitWolf': <pyspi.statistics.basic.Covari

In [40]:
basic_spis = ['Covariance', 'Precision', 'SpearmanR', 
                'KendallTau', 'CrossCorrelation']
for spi_name in basic_spis:
    class_to_check = getattr(pyspi.statistics.basic, spi_name)
    print(class_to_check)
    spis_all = [key for key, value in spi_dict.items() if isinstance(value, class_to_check)]
    print(spis_all)

<class 'pyspi.statistics.basic.Covariance'>
['cov_EmpiricalCovariance', 'cov_EllipticEnvelope', 'cov_GraphicalLasso', 'cov_GraphicalLassoCV', 'cov_LedoitWolf', 'cov_MinCovDet', 'cov_OAS', 'cov_ShrunkCovariance', 'cov-sq_EmpiricalCovariance', 'cov-sq_EllipticEnvelope', 'cov-sq_GraphicalLasso', 'cov-sq_GraphicalLassoCV', 'cov-sq_LedoitWolf', 'cov-sq_MinCovDet', 'cov-sq_OAS', 'cov-sq_ShrunkCovariance']
<class 'pyspi.statistics.basic.Precision'>
['prec_EmpiricalCovariance', 'prec_EllipticEnvelope', 'prec_GraphicalLasso', 'prec_GraphicalLassoCV', 'prec_LedoitWolf', 'prec_MinCovDet', 'prec_OAS', 'prec_ShrunkCovariance', 'prec-sq_EmpiricalCovariance', 'prec-sq_EllipticEnvelope', 'prec-sq_GraphicalLasso', 'prec-sq_GraphicalLassoCV', 'prec-sq_LedoitWolf', 'prec-sq_MinCovDet', 'prec-sq_OAS', 'prec-sq_ShrunkCovariance']
<class 'pyspi.statistics.basic.SpearmanR'>
['spearmanr-sq', 'spearmanr']
<class 'pyspi.statistics.basic.KendallTau'>
['kendalltau-sq', 'kendalltau']
<class 'pyspi.statistics.basic

In [39]:
pyspi.statistics.basic.Covariance.identifier

'cov'

In [26]:
[key for key, value in spi_dict.items() if isinstance(value, pyspi.statistics.basic.Precision)]

['prec_EmpiricalCovariance',
 'prec_EllipticEnvelope',
 'prec_GraphicalLasso',
 'prec_GraphicalLassoCV',
 'prec_LedoitWolf',
 'prec_MinCovDet',
 'prec_OAS',
 'prec_ShrunkCovariance',
 'prec-sq_EmpiricalCovariance',
 'prec-sq_EllipticEnvelope',
 'prec-sq_GraphicalLasso',
 'prec-sq_GraphicalLassoCV',
 'prec-sq_LedoitWolf',
 'prec-sq_MinCovDet',
 'prec-sq_OAS',
 'prec-sq_ShrunkCovariance']

In [22]:
calc.table['cov_EllipticEnvelope']

process,proc-0,proc-1,proc-2,proc-3,proc-4
proc-0,NaN,0.076453,-0.168245,0.108380,0.005180
proc-1,0.076453,NaN,-0.157841,-0.139266,-0.141852
proc-2,-0.168245,-0.157841,NaN,0.053996,-0.046680
proc-3,0.108380,-0.139266,0.053996,NaN,0.147468
proc-4,0.005180,-0.141852,-0.046680,0.147468,NaN


In [42]:
def load_benchmark_calcs():
    benchmark_calcs = dict()
    calcs = ['calc_standard_normal.pkl'] # follow this naming convention -> calc_{name}.pkl
    for calc in calcs:
            # extract the calculator name from the filename
            calc_name = calc[len("calc_"):-len(".pkl")]

            # Load the calculator
            with open(f"benchmark_calculators/{calc}", "rb") as f:
                loaded_calc = dill.load(f)
                benchmark_calcs[calc_name] = loaded_calc

    return benchmark_calcs

#@pytest.fixture
def load_benchmark_datasets():
    benchmark_datasets = dict()
    dataset_names = ['standard_normal.npy']
    for dname in dataset_names:
        dataset = np.load(f"../pyspi/data/{dname}")
        dataset = dataset.T 
        benchmark_datasets[dname.strip('.npy')] = dataset

    return benchmark_datasets


def compute_new_tables():
    # Compute new tables on the benchmark datasets
    benchmark_datasets = load_benchmark_datasets()
    new_calcs = dict()
    calc_base = Calculator() # create base calculator object
    for dataset in benchmark_datasets.keys():
        calc = deepcopy(calc_base) # make a copy of the base calculator
        calc.load_dataset(dataset=benchmark_datasets[dataset])
        calc.compute()
        new_calcs[dataset] = calc

    return new_calcs

In [46]:
benchmark_calcs = load_benchmark_calcs()
benchmark_calcs

{'standard_normal': <pyspi.calculator.Calculator at 0x7f8469290d00>}

In [47]:
benchmark_datasets = load_benchmark_datasets()
benchmark_datasets

{'standard_normal': array([[ 4.96714153e-01, -1.38264301e-01,  6.47688538e-01,
          1.52302986e+00, -2.34153375e-01, -2.34136957e-01,
          1.57921282e+00,  7.67434729e-01, -4.69474386e-01,
          5.42560044e-01, -4.63417693e-01, -4.65729754e-01,
          2.41962272e-01, -1.91328024e+00, -1.72491783e+00,
         -5.62287529e-01, -1.01283112e+00,  3.14247333e-01,
         -9.08024076e-01, -1.41230370e+00,  1.46564877e+00,
         -2.25776300e-01,  6.75282047e-02, -1.42474819e+00,
         -5.44382725e-01,  1.10922590e-01, -1.15099358e+00,
          3.75698018e-01, -6.00638690e-01, -2.91693750e-01,
         -6.01706612e-01,  1.85227818e+00, -1.34972247e-02,
         -1.05771093e+00,  8.22544912e-01, -1.22084365e+00,
          2.08863595e-01, -1.95967012e+00, -1.32818605e+00,
          1.96861236e-01,  7.38466580e-01,  1.71368281e-01,
         -1.15648282e-01, -3.01103696e-01, -1.47852199e+00,
         -7.19844208e-01, -4.60638771e-01,  1.05712223e+00,
          3.43618290e

In [49]:
new_calcs = compute_new_tables()

Frequency minimum set to 0; overriding to 1e-5.
Processing [None: cov_GraphicalLassoCV]:   0%|          | 0/283 [00:00<?, ?it/s]   

Loading configuration file: /Users/joshua/opt/anaconda3/envs/pyspi/lib/python3.9/site-packages/pyspi/config.yaml
*** Importing module .statistics.basic
[0] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EmpiricalCovariance'})...
Succesfully initialised SPI with identifier "cov_EmpiricalCovariance" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[1] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EllipticEnvelope'})...
Succesfully initialised SPI with identifier "cov_EllipticEnvelope" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[2] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLasso'})...
Succesfully initialised SPI with identifier "cov_GraphicalLasso" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[3] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLassoCV'})...
Succesfully initialised SPI with identifier "cov_GraphicalLassoCV" and labels ['basic', 'unordere

Processing [None: phi_star_t-1_norm-0]:  46%|████▌     | 130/283 [02:09<00:26,  5.72it/s]                   

Processing [None: phi_star_t-1_norm-0]:  46%|████▌     | 130/283 [02:22<00:26,  5.72it/s]

Processing [None: phi_star_t-1_norm-1]:  48%|████▊     | 136/283 [03:17<08:36,  3.51s/it]

Processing [None: phi_Geo_t-1_norm-0]:  48%|████▊     | 137/283 [04:20<52:06, 21.42s/it] 

Processing [None: phi_Geo_t-1_norm-1]:  49%|████▉     | 138/283 [05:23<1:22:15, 34.04s/it]

Processing [None: phase_multitaper_mean_fs-1_fmin-0_fmax-0-5]:  49%|████▉     | 139/283 [06:25<1:41:47, 42.41s/it]Mean of empty slice
Processing [None: phase_multitaper_max_fs-1_fmin-0_fmax-0-5]:  49%|████▉     | 139/283 [06:25<1:41:47, 42.41s/it]    All-NaN slice encountered
Processing [None: sgc_nonparametric_mean_fs-1_fmin-0_fmax-0-5]:  79%|███████▉  | 223/283 [06:27<07:09,  7.15s/it]     Mean of empty slice
Processing [None: sgc_nonparametric_max_fs-1_fmin-0_fmax-0-5]:  79%|███████▉  | 223/283 [06:27<07:09,  7.15s/it]    All-NaN slice encountered
Processing [None: pec_orth_log_abs]: 100%|██████████| 283/283 [06:30<00:00,  1.38s/it]                                      


In [56]:
for calc_name, calc in new_calcs.items():
    print(calc)

In [ ]:
pyspi.statistics.basic.getattr

In [63]:
new_calcs['standard_normal'].table['cov_EmpiricalCovariance']

process,proc-0,proc-1,proc-2,proc-3,proc-4
proc-0,NaN,0.095244,-0.132890,0.065728,-0.025382
proc-1,0.095244,NaN,-0.032535,-0.100964,-0.141970
proc-2,-0.132890,-0.032535,NaN,0.105346,0.025010
proc-3,0.065728,-0.100964,0.105346,NaN,0.169568
proc-4,-0.025382,-0.141970,0.025010,0.169568,NaN


In [68]:
calc.table['cov_EmpiricalCovariance'].equals(new_calcs['standard_normal'].table['cov_EmpiricalCovariance'])

True

In [77]:
def test_basic_spis(benchmark_calcs, new_calcs):
    # test each of the basic SPIs on each of the datasets
    basic_spis = ['Covariance', 'Precision', 'SpearmanR', 
                'KendallTau', 'CrossCorrelation']
    
    for calc_name, benchmark_calc in benchmark_calcs.items():

        # get all SPI names for calculator
        spi_dict = benchmark_calc.spis

        # loop through each calculator and test the basic SPIs
        print(f'Testing {calc_name}...')
        # loop through each SPI
        for spi_name in basic_spis:

            print(f'Checking {spi_name}...')
            spi_to_check = getattr(pyspi.statistics.basic, spi_name)
            spi_estimators = [key for key, value in spi_dict.items() if isinstance(value, spi_to_check)]

            # now for each spi estimator compare the new table to the reference (benchmark) table
            for est in spi_estimators:

                mpi_benchmark = benchmark_calc.table[est]
                mpi_new = new_calcs[calc_name].table[est]

                # now compare the two tables
                if not mpi_benchmark.equals(mpi_new):
                    # compute the abs diff
                    diff = abs(mpi_benchmark - mpi_new)
                    
                    # get total number of differences
                    total_differences = diff.values.size - np.count_nonzero(diff.isna().values)
                    min_diff = diff.min().min()
                    max_diff = diff.max().max()

                    assert False, f"SPI: {est} gave a different output on the {calc_name} dataset!\n " \
                                  f"Total differences: {total_differences}, " \
                                  f"Min difference: {min_diff}, Max difference: {max_diff}"          

In [78]:
test_basic_spis(benchmark_calcs, new_calcs)

Testing standard_normal...
Checking Covariance...


AssertionError: SPI: cov_EllipticEnvelope gave a different output on the standard_normal dataset!
 Total differences: 20, Min difference: 0.00028642995530295257, Max difference: 0.01841191658818296

In [83]:
def generate_test_params(benchmark_calcs, new_calcs, basic_spis):
    params = []
    for calc_name, benchmark_calc in benchmark_calcs.items():
        spi_dict = benchmark_calc.spis
        for spi_name in basic_spis:
            spi_to_check = getattr(pyspi.statistics.basic, spi_name)
            spi_estimators = [key for key, value in spi_dict.items() if isinstance(value, spi_to_check)]
            for est in spi_estimators:
                params.append((calc_name, est, benchmark_calc.table[est], new_calcs[calc_name].table[est]))
    return params

In [102]:
import pytest

In [85]:
basic_spis = ['Covariance', 'Precision', 'SpearmanR', 
                'KendallTau', 'CrossCorrelation']
params = generate_test_params(benchmark_calcs, new_calcs, basic_spis)

In [101]:
params[0][3]

process,proc-0,proc-1,proc-2,proc-3,proc-4
proc-0,NaN,0.095244,-0.132890,0.065728,-0.025382
proc-1,0.095244,NaN,-0.032535,-0.100964,-0.141970
proc-2,-0.132890,-0.032535,NaN,0.105346,0.025010
proc-3,0.065728,-0.100964,0.105346,NaN,0.169568
proc-4,-0.025382,-0.141970,0.025010,0.169568,NaN


In [103]:
# Parameterized test function
@pytest.mark.parametrize("calc_name,est,mpi_benchmark,mpi_new", generate_test_params(load_benchmark_calcs(), compute_new_tables(), ['Covariance', 'Precision', 'SpearmanR', 'KendallTau', 'CrossCorrelation']))
def test_mpi(calc_name, est, mpi_benchmark, mpi_new):
    if not mpi_benchmark.equals(mpi_new):
        diff = abs(mpi_benchmark - mpi_new)
        total_differences = diff.values.size - np.count_nonzero(diff.isna().values)
        min_diff = diff.min().min()
        max_diff = diff.max().max()
        assert False, f"SPI: {est} gave a different output on the {calc_name} dataset!\n" \
                      f"Total differences: {total_differences}, " \
                      f"Min difference: {min_diff}, Max difference: {max_diff}"

Frequency minimum set to 0; overriding to 1e-5.
Processing [None: cov_GraphicalLassoCV]:   0%|          | 0/283 [00:00<?, ?it/s]   

Loading configuration file: /Users/joshua/opt/anaconda3/envs/pyspi/lib/python3.9/site-packages/pyspi/config.yaml
*** Importing module .statistics.basic
[0] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EmpiricalCovariance'})...
Succesfully initialised SPI with identifier "cov_EmpiricalCovariance" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[1] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EllipticEnvelope'})...
Succesfully initialised SPI with identifier "cov_EllipticEnvelope" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[2] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLasso'})...
Succesfully initialised SPI with identifier "cov_GraphicalLasso" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[3] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLassoCV'})...
Succesfully initialised SPI with identifier "cov_GraphicalLassoCV" and labels ['basic', 'unordere

Processing [None: phi_star_t-1_norm-0]:  47%|████▋     | 133/283 [01:56<00:27,  5.49it/s]                   

Processing [None: phi_star_t-1_norm-0]:  47%|████▋     | 133/283 [02:07<00:27,  5.49it/s]

Processing [None: phi_star_t-1_norm-1]:  48%|████▊     | 136/283 [03:00<15:54,  6.49s/it]

Processing [None: phi_Geo_t-1_norm-0]:  48%|████▊     | 137/283 [04:03<57:25, 23.60s/it] 

Processing [None: phi_Geo_t-1_norm-1]:  49%|████▉     | 138/283 [05:07<1:25:51, 35.53s/it]

Processing [None: phase_multitaper_mean_fs-1_fmin-0_fmax-0-5]:  49%|████▉     | 139/283 [06:09<1:44:53, 43.70s/it]Mean of empty slice
Processing [None: phase_multitaper_max_fs-1_fmin-0_fmax-0-5]:  49%|████▉     | 139/283 [06:09<1:44:53, 43.70s/it]    All-NaN slice encountered
Processing [None: sgc_nonparametric_mean_fs-1_fmin-0_fmax-0-5]:  78%|███████▊  | 221/283 [06:11<07:36,  7.37s/it]     Mean of empty slice
Processing [None: sgc_nonparametric_max_fs-1_fmin-0_fmax-0-5]:  78%|███████▊  | 221/283 [06:11<07:36,  7.37s/it]    All-NaN slice encountered
Processing [None: pec_orth_log_abs]: 100%|██████████| 283/283 [06:13<00:00,  1.32s/it]                                      


In [106]:
@pytest.fixture
def load_benchmark_calcs():
    benchmark_calcs = dict()
    calcs = ['calc_standard_normal.pkl'] # follow this naming convention -> calc_{name}.pkl
    for calc in calcs:
            # extract the calculator name from the filename
            calc_name = calc[len("calc_"):-len(".pkl")]

            # Load the calculator
            with open(f"../pyspi/tests/benchmark_calculators/{calc}", "rb") as f:
                loaded_calc = dill.load(f)
                benchmark_calcs[calc_name] = loaded_calc

    return benchmark_calcs

@pytest.fixture
def load_benchmark_datasets():
    benchmark_datasets = dict()
    dataset_names = ['standard_normal.npy']
    for dname in dataset_names:
        dataset = np.load(f"pyspi/data/{dname}")
        dataset = dataset.T 
        benchmark_datasets[dname.strip('.npy')] = dataset

    return benchmark_datasets

@pytest.fixture
def compute_new_tables(load_benchmark_datasets):
    benchmark_datasets = load_benchmark_datasets
    # Compute new tables on the benchmark datasets
    new_calcs = dict()
    calc_base = Calculator() # create base calculator object
    for dataset in benchmark_datasets.keys():
        calc = deepcopy(calc_base) # make a copy of the base calculator
        calc.load_dataset(dataset=benchmark_datasets[dataset])
        calc.compute()
        new_calcs[dataset] = calc

    return new_calcs

@pytest.fixture
def generate_test_params(load_benchmark_datasets, compute_new_tables):
    benchmark_calcs = load_benchmark_datasets
    new_calcs = compute_new_tables
    basic_spis = ['Covariance', 'Precision', 'SpearmanR', 'KendallTau', 'CrossCorrelation']
    params = []
    for calc_name, benchmark_calc in benchmark_calcs.items():
        spi_dict = benchmark_calc.spis
        for spi_name in basic_spis:
            spi_to_check = getattr(pyspi.statistics.basic, spi_name)
            spi_estimators = [key for key, value in spi_dict.items() if isinstance(value, spi_to_check)]
            for est in spi_estimators:
                params.append((calc_name, est, benchmark_calc.table[est], new_calcs[calc_name].table[est]))
    return params

@pytest.mark.parametrize("calc_name,est,mpi_benchmark,mpi_new", generate_test_params)
def test_mpi(calc_name, est, mpi_benchmark, mpi_new):
    if not mpi_benchmark.equals(mpi_new):
        diff = abs(mpi_benchmark - mpi_new)
        total_differences = diff.values.size - np.count_nonzero(diff.isna().values)
        min_diff = diff.min().min()
        max_diff = diff.max().max()
        assert False, f"SPI: {est} gave a different output on the {calc_name} dataset!\n" \
                      f"Total differences: {total_differences}, " \
                      f"Min difference: {min_diff}, Max difference: {max_diff}"

In [113]:
def generate_test_params(new_calcs, benchmark_calcs):
    # Load your data
    basic_spis = ['Covariance', 'Precision', 'SpearmanR', 'KendallTau', 'CrossCorrelation']
    params = []
    for calc_name, benchmark_calc in benchmark_calcs.items():
        spi_dict = benchmark_calc.spis
        for spi_name in basic_spis:
            spi_to_check = getattr(pyspi.statistics.basic, spi_name)
            spi_estimators = [key for key, value in spi_dict.items() if isinstance(value, spi_to_check)]
            for est in spi_estimators:
                params.append((calc_name, est, benchmark_calc.table[est], new_calcs[calc_name].table[est]))
    return params

In [119]:
params = generate_test_params(new_calcs, benchmark_calcs)
@pytest.mark.parametrize("calc_name,est,mpi_benchmark,mpi_new", params)
def test_mpi(calc_name, est, mpi_benchmark, mpi_new):
    if not mpi_benchmark.equals(mpi_new):
        diff = abs(mpi_benchmark - mpi_new)
        total_differences = diff.values.size - np.count_nonzero(diff.isna().values)
        min_diff = diff.min().min()
        max_diff = diff.max().max()
        assert False, f"SPI: {est} gave a different output on the {calc_name} dataset!\n" \
                      f"Total differences: {total_differences}, " \
                      f"Min difference: {min_diff}, Max difference: {max_diff}"